Back Tracing is the processing of reverse engineering a function back to the original formula. Most researchers do this to understand which part of the mathematical foundations of the algorithmic function can be further improved.

Example, you are given a code:

In [1]:
def euclidean_distance(x1,x2,y1,y2):
    z = ((x1-x2)**2+(y1-y2)**2)**(0.5)
    return z

The formula above is a function for the Euclidean distance, back tracing (reverse engineering) it will yield:

$$
z = \sqrt{(x_1-x_2)^2 + (y_1 - y_2)^2}
$$

In [ ]:
## Begin Actual Backtrace ##

I shall be backtracing the functions Comprising the Multinomial Expectation Maximizer that utilizes essential mathematical formulas for a Multinomial Mixture Model

First off , The MultinomialExpectationMaximizer object is initialized using the following code :

In [ ]:
model = MultinomialExpectationMaximizer(k, restarts=10, rtol=1e-3)

def __init__(self, K, rtol=1e-3, max_iter=100, restarts=10):
        self._K = K
        self._rtol = rtol
        self._max_iter = max_iter
        self._restarts = restarts

The parameters are as follows :

k: represents the number of mixture components 

rtol=1e-3 : Sets the relative tolerance for convergence that will stop the algorithm when the change in log-likelihood between iterations is less than this value.

max_iter=100 : Sets the maximum number of iterations for the EM algorithm

restarts=10 : Sets the number of times the EM algorithm will be restarted with random initializations to avoid getting stuck in local optima and finding a better overall solution

In [ ]:
def _train_once(self, X):
        '''
        Runs one full cycle of the EM algorithm

        :param X: (N, C), matrix of counts
        :return: The best parameters found along with the associated loss
        '''
        loss = float('inf')
        alpha, beta = self._init_params(X)
        gamma = None

        for it in range(self._max_iter):
            prev_loss = loss
            gamma = self._e_step(X, alpha, beta)
            alpha, beta = self._m_step(X, gamma)
            loss = self._compute_vlb(X, alpha, beta, gamma)
            print('Loss: %f' % loss)
            if it > 0 and np.abs((prev_loss - loss) / prev_loss) < self._rtol:
                    break
        return alpha, beta, gamma, loss

def _train_once

the function runs a single iteration of the EM algorithm for training the model and returns the best parameters alpha and beta, the latent variables gamma, and the associated loss. The breakdown of the function is as follows :

- It initializes the loss to infinity and calls the _init_params method to initialize the parameters alpha and beta based on the input X.
- It sets the latent variables gamma to None.
- It iterates over a maximum number of iterations _max_iter
- In each iteration, it does the following:
  1. It saves the previous loss value.
  2. It calls the _e_step method to perform the expectation step of the EM algorithm
  3. It calls the _m_step method to perform the maximization step of the EM algorithm
  4. It calls the _compute_vlb method to compute the variational lower bound (VLB) of the log-likelihood
  5. It prints the current loss value.
  6. It checks if the relative change in the loss value is smaller than a tolerance _rtol, and If so, it breaks the loop and stops the iteration.
- It returns the final values of alpha, beta, gamma, and loss.

Here are the following formulas used in this function :

Weighted multinomial (posterior) probability for object i and cluster k (weighted_multi_prob[i, k] / P(k|X_i)):
$$P(k|X_i) ∝ P(X_i|k) * P(k)$$

Denominator for normalization of posterior probabilities for object i (denum) :
$$denum_i = sum(P(k|X_i) for k in K)$$

Posterior probability of object i belonging to cluster k (gamma/P(k|X_i)):
$$P(k|X_i) = P(X_i|k) * P(k) / sum(P(j|X_i) for j in K)$$

In [ ]:
def _m_step(self, X, gamma):
        """
        Performs M-step on MNMM model
        Each input is numpy array:
        X: (N x C), matrix of counts
        gamma: (N x K), posterior probabilities for objects clusters assignments

        Returns:
        alpha: (K), mixture component weights
        beta: (K x C), mixture categories weights
        """
        # Compute alpha
        alpha = self._m_step_alpha(gamma)

        # Compute beta
        beta = self._m_step_beta(X, gamma)

        return alpha, beta

    def _m_step_alpha(self, gamma):
        alpha = gamma.sum(axis=0) / gamma.sum()
        return alpha

    def _m_step_beta(self, X, gamma):
        weighted_counts = gamma.T.dot(X)
        beta = weighted_counts / weighted_counts.sum(axis=-1).reshape(-1, 1)
        return beta

def _m_step

This code implements the M-step of the model  to update the model parameters (alpha and beta) based on the posterior probabilities (gamma) estimated in the E-step by calling m_step_alpha and m_step_beta.

Updated mixture weights (alpha):
$$α_k = ∑_i^N γ_ik / ∑_i^N ∑_k^K γ_ik$$

Updated multinomial weights (beta): 
$$β_kc = ∑_i^N γ_ik * x_ic / ∑_j^C ∑_i^N γ_ik * x_ij$$

In [ ]:
def _compute_vlb(self, X, alpha, beta, gamma):
        """
        Computes the variational lower bound
        X: (N x C), data points
        alpha: (K) or (NxK) with individual weights, mixture component weights
        beta: (K x C), multinomial categories weights
        gamma: (N x K), posterior probabilities for objects clusters assignments

        Returns value of variational lower bound
        """
        loss = 0
        for k in range(beta.shape[0]):
            weights = gamma[:, k]
            loss += np.sum(weights * (np.log(self._get_mixture_weight(alpha, k)) + self._multinomial_prob(X, beta[k], log=True)))
            loss -= np.sum(weights * np.log(weights))
        return loss

def _compute_vlb

This code calculates the VLB (loss) for the model that will be used in variational inference to approximate model parameters.

$$VLB = ∑_k ∑_i γ_ik * (log(α_k) + log(p(X_i|k))) - ∑_k ∑_i γ_ik * log(γ_ik)$$

In [ ]:
def _multinomial_prob(self, counts, beta, log=False):
        """
        Evaluates the multinomial probability for a given vector of counts
        counts: (N x C), matrix of counts
        beta: (C), vector of multinomial parameters for a specific cluster k
        Returns:
        p: (N), scalar values for the probabilities of observing each count vector given the beta parameters
        """
        n = counts.sum(axis=-1)
        m = multinomial(n, beta)
        if log:
            return m.logpmf(counts)
        return m.pmf(counts)

def _multinomial_prob

This code computes the multinomial probability of observing a given vector of counts with specified parameters (beta)

$$P(X|k) = (n! / (x1! * x2! * ... * xC!)) * (beta_k1^x1 * beta_k2^x2 * ... * beta_kC^xC)$$

In [ ]:
def compute_log_likelihood(self, X_test, alpha, beta):
        mn_probs = np.zeros(X_test.shape[0])
        for k in range(beta.shape[0]):
            mn_probs_k = self._get_mixture_weight(alpha, k) * self._multinomial_prob(X_test, beta[k])
            mn_probs += mn_probs_k
        mn_probs[mn_probs == 0] = np.finfo(float).eps
        return np.log(mn_probs).sum()

def compute_log_likelihood

the function calculates the log-likelihood (Log(L)) of a given dataset (X_test) with specified parameters (alpha and beta).

$$log(L) = ∑_i log(∑_k α_k * p(X_i|k))$$

In [ ]:
def compute_bic(self, X_test, alpha, beta, log_likelihood=None):
        if log_likelihood is None:
            log_likelihood = self.compute_log_likelihood(X_test, alpha, beta)
        N = X_test.shape[0]
        return np.log(N) * (alpha.size + beta.size) - 2 * log_likelihood

def compute_bic

 Calculates the BIC given model parameters (alpha and beta) and a dataset (X_test)

$$BIC = log(N) * d - 2 * log(L)$$

In [ ]:
def compute_icl_bic(self, bic, gamma):
        classification_entropy = -(np.log(gamma.max(axis=1))).sum()
        return bic - classification_entropy

def compute_icl_bic 

calculates the ICL-BIC, a variant of the BIC that incorporates a penalty for model complexity based on cluster assignments

$$ICL = BIC - ∑_i log(max_k γ_ik)$$